In [8]:
from flask import Flask,request
from flask import Flask
from flask import jsonify, request
import shutil
import os
from werkzeug.utils import secure_filename
import json
import traceback
import sys
import cv2
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

In [9]:
def aHash(img, size):
    # 均值哈希算法
    # 縮放爲8*8
    img = cv2.resize(img, size)
    # 轉換爲灰度圖
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # s爲像素和初值爲0，hash_str爲hash值初值爲''
    s = 0
    hash_str = ''
    # 遍歷累加求像素和
    for i in range(size[0]):
        for j in range(size[1]):
            s = s+gray[i, j]
    # 求平均灰度
    avg = s/(size[0]*size[1])
    # 灰度大於平均值爲1相反爲0生成圖片的hash值
    for i in range(size[0]):
        for j in range(size[1]):
            if gray[i, j] > avg:
                hash_str = hash_str+'1'
            else:
                hash_str = hash_str+'0'
    return hash_str


def dHash(img, size):
    # 差值哈希算法
    # 縮放8*8
    img = cv2.resize(img, (size[0]+1,size[1]))
    # 轉換灰度圖
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hash_str = ''
    # 每行前一個像素大於後一個像素爲1，相反爲0，生成哈希
    for i in range(size[0]):
        for j in range(size[1]):
            if gray[i, j] > gray[i, j+1]:
                hash_str = hash_str+'1'
            else:
                hash_str = hash_str+'0'
    return hash_str


def pHash(img, size):
    # 感知哈希算法
    # 縮放32*32
    img = cv2.resize(img, size)   # , interpolation=cv2.INTER_CUBIC

    # 轉換爲灰度圖
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    # 將灰度圖轉爲浮點型，再進行dct變換
    dct = cv2.dct(np.float32(gray))
    # opencv實現的掩碼操作
    dct_roi = dct[0:8, 0:8]

    hash = []
    avreage = np.mean(dct_roi)
    for i in range(dct_roi.shape[0]):
        for j in range(dct_roi.shape[1]):
            if dct_roi[i, j] > avreage:
                hash.append(1)
            else:
                hash.append(0)
    return hash


def calculate(image1, image2,size):
    # 灰度直方圖算法
    # 計算單通道的直方圖的相似值
    image1 = cv2.resize(image1, size)
    image2 = cv2.resize(image2, size)
    hist1 = cv2.calcHist([image1], [0], None, [max(size)], [0.0, 255.0])
    hist2 = cv2.calcHist([image2], [0], None, [max(size)], [0.0, 255.0])
    # 計算直方圖的重合度
    degree = 0
    for i in range(len(hist1)):
        if hist1[i] != hist2[i]:
            degree = degree + \
                (1 - abs(hist1[i] - hist2[i]) / max(hist1[i], hist2[i]))
        else:
            degree = degree + 1
    degree = degree / len(hist1)
    return degree


def classify_hist_with_split(image1, image2, size):
    # RGB每個通道的直方圖相似度
    # 將圖像resize後，分離爲RGB三個通道，再計算每個通道的相似值
    image1 = cv2.resize(image1, size)
    image2 = cv2.resize(image2, size)
    sub_image1 = cv2.split(image1)
    sub_image2 = cv2.split(image2)
    sub_data = 0
    for im1, im2 in zip(sub_image1, sub_image2):
        sub_data += calculate(im1, im2, size)
    sub_data = sub_data / 3
    return sub_data


def cmpHash(hash1, hash2):
    # Hash值對比
    # 算法中1和0順序組合起來的即是圖片的指紋hash。順序不固定，但是比較的時候必須是相同的順序。
    # 對比兩幅圖的指紋，計算漢明距離，即兩個64位的hash值有多少是不一樣的，不同的位數越小，圖片越相似
    # 漢明距離：一組二進制數據變成另一組數據所需要的步驟，可以衡量兩圖的差異，漢明距離越小，則相似度越高。漢明距離爲0，即兩張圖片完全一樣
    n = 0
    # hash長度不同則返回-1代表傳參出錯
    if len(hash1) != len(hash2):
        return -1
    # 遍歷判斷
    for i in range(len(hash1)):
        # 不相等則n計數+1，n最終爲相似度
        if hash1[i] != hash2[i]:
            n = n + 1
    return n


def getImageByUrl(url):
    # 根據圖片url 獲取圖片對象
    html = requests.get(url, verify=False)
    image = Image.open(BytesIO(html.content))
    return image


def PILImageToCV():
    # PIL Image轉換成OpenCV格式
    path = "/Users/waldenz/Documents/Work/doc/TestImages/t3.png"
    img = Image.open(path)
    plt.subplot(121)
    plt.imshow(img)
    print(isinstance(img, np.ndarray))
    img = cv2.cvtColor(np.asarray(img), cv2.COLOR_RGB2BGR)
    print(isinstance(img, np.ndarray))
    plt.subplot(122)
    plt.imshow(img)
    plt.show()


def CVImageToPIL():
    # OpenCV圖片轉換爲PIL image
    path = "/Users/waldenz/Documents/Work/doc/TestImages/t3.png"
    img = cv2.imread(path)
    # cv2.imshow("OpenCV",img)
    plt.subplot(121)
    plt.imshow(img)

    img2 = Image.fromarray(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
#     plt.subplot(122)
#     plt.imshow(img2)
#     plt.show()

def bytes_to_cvimage(filebytes):
    # 圖片字節流轉換爲cv image
    image = Image.open(filebytes)
    image.close() 
    img = cv2.cvtColor(np.asarray(image), cv2.COLOR_RGB2BGR)
    return img

def runAllImageSimilaryFun(para1, para2, size=(256,256)):
    ans={}
    # 均值、差值、感知哈希算法三種算法值越小，則越相似,相同圖片值爲0
    # 三直方圖算法和單通道的直方圖 0-1之間，值越大，越相似。 相同圖片爲1

    # t1,t2   14;19;10;  0.70;0.75
    # t1,t3   39 33 18   0.58 0.49
    # s1,s2  7 23 11     0.83 0.86  挺相似的圖片
    # c1,c2  11 29 17    0.30 0.31

    if para1.startswith("http"):
         # 根據鏈接下載圖片，並轉換爲opencv格式
        img1 = getImageByUrl(para1)
        img1 = cv2.cvtColor(np.asarray(img1), cv2.COLOR_RGB2BGR)

        img2 = getImageByUrl(para2)
        img2 = cv2.cvtColor(np.asarray(img2), cv2.COLOR_RGB2BGR)
    else:
        # 通過imread方法直接讀取物理路徑
        img1 = cv2.imread(para1)
        img2 = cv2.imread(para2)

    hash1 = aHash(img1, size)
    hash2 = aHash(img2, size)
    n1 = cmpHash(hash1, hash2)
    ans['均值哈希算法']=1-(n1/(size[0]*size[1]))


    hash1 = dHash(img1, size)
    hash2 = dHash(img2, size)
    n2 = cmpHash(hash1, hash2)
    ans['差值哈希算法']=1-(n2/(size[0]*size[1]))


    hash1 = pHash(img1, size)
    hash2 = pHash(img2, size)
    n3 = cmpHash(hash1, hash2)
    ans['感知哈希算法']=1-(n3/(size[0]*size[1]))


    n4 = classify_hist_with_split(img1, img2, size)
    try:
        ans['三直方圖算法']=n4[0]
    except:
        ans['三直方圖算法']=n4


    n5 = calculate(img1, img2, size)
    try:
        ans['單通道的直方圖']=n5[0]
    except:
        ans['單通道的直方圖']=n5
    
    return ans

In [16]:
my_host=input("請輸入主機ip ( 若要從本機啟動，可輸入127.0.0.1 )：")
port = input("請輸入欲開放之端口 ( 可輸入5000 )：")
ALLOWED_EXTENSIONS = set(['txt', 'pdf', 'png', 'jpg', 'jpeg', 'gif'])

app = Flask(__name__)
@app.route('/', methods=['GET'])
def cc():
    return "參數帶 p1=C:\\1.png&p2=C:\\2.png&size=100，範例：http://127.0.0.1:5000/upload?p1=C:\\1.png&p2=C:\\2.png&size=100，size參數可以不用帶，用來定義圖片縮放大小的",200

@app.route("/upload", methods=["GET"])
def upload():
    if request.args:
        dd= request.args.to_dict()
        if set(['p1','p2','size']) >= set(dd.keys()):
            print("傳參成功")
        else:
            return "傳參失敗，參數只允許"+' p1 '+'p2 '+'size ',500
#     path = './uploads'
#     shutil.rmtree (path)
#     os.mkdir(path)
#     uploaded_files = request.files.getlist("file")
#     file = uploaded_files[0]
#     filename = secure_filename(file.filename)
#     file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
#     file = uploaded_files[1]
#     filename = secure_filename(file.filename)
#     file.save(os.path.join(app.config['UPLOAD_FOLDER'], filename))
    
    try:
        p1 = str(request.args.get("p1"))
    except:
        return "p1參數轉換失敗",500
    try:
        p2 = str(request.args.get("p2"))
    except:
        return "p2參數轉換失敗",500
        
    try:
        if request.args.get("size"):
            try:
                arg_size = int(request.args.get("size"))
            except:
                return "size參數錯誤",500
            size=[arg_size,arg_size]
            xx = runAllImageSimilaryFun(p1,p2,size)
        else:
            xx = runAllImageSimilaryFun(p1,p2)
        print(xx)
        return json.dumps(str(xx)),200
    except Exception as ex:
        print(ex)
        error_class = ex.__class__.__name__ #取得錯誤類型
        detail = ex.args[0] #取得詳細內容
        cl, exc, tb = sys.exc_info() #取得Call Stack
        lastCallStack = traceback.extract_tb(tb)[-1] #取得Call Stack的最後一筆資料
        fileName = lastCallStack[0] #取得發生的檔案名稱
        lineNum = lastCallStack[1] #取得發生的行號
        funcName = lastCallStack[2] #取得發生的函數名稱
        errMsg = "File \"{}\", line {}, in {}: [{}] {}".format(fileName, lineNum, funcName, error_class, detail)
        return errMsg,500

if __name__ == '__main__':
    app.debug = False
    app.config['JSON_AS_ASCII'] = False
    app.run(host=my_host, port=port)

請輸入主機ip ( 若要從本機啟動，可輸入127.0.0.1 )：
請輸入欲開放之端口 ( 可輸入5000 )：5000
 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [13/Feb/2022 20:52:02] "GET /upload?p1=C:\1.png&p2=C:\2.png&size=[599,600] HTTP/1.1" 500 -


傳參成功
傳參成功


127.0.0.1 - - [13/Feb/2022 20:52:06] "GET /upload?p1=C:\1.png&p2=C:\2.png&size=599 HTTP/1.1" 200 -


{'均值哈希算法': 0.7530358053628613, '差值哈希算法': 0.898598387406947, '感知哈希算法': 0.9999804905783428, '三直方圖算法': 0.7659829, '單通道的直方圖': 0.8285072}
